# Data Preparation

For our data, we will be collecting from two sources, and combining them for a single dataset.

The first is a dataset of the English Premier League collected by Tara Nguyen in Kaggle from 2010 to 2018. This data set is a match-by-match statistical report (similar to above) of every season from 2010/11 to 2017/2018. As explained, there are 380 matches * 8 seasons = 3040 matches (or rows) in our dataset.

As Nguyen's data set lacked cetain match data that are important to our project, we created another dataset through web scraping from the official Premier League website (www.premierleague.com) and appended the data to the current dataset.

First, lets import Tara Nguyen's dataset

In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
sb.set() 

In [3]:
all_seasons_data = pd.read_csv("https://raw.githubusercontent.com/tara-nguyen/english-premier-league-datasets-for-10-seasons/main/epldat10seasons/epl-allseasons-matchstats.csv")
# Exclude the 2018/19 and 2019/20 seasons
values = ['2018/19', '2019/20']
all_seasons_data = all_seasons_data[all_seasons_data.Season.isin(values) == False]
all_seasons_data

,Season,Date,Referee,HomeTeam,AwayTeam,FullTime,Halftime,HomeGoals,HomeGoalsHalftime,HomeShots,...,HomeYellowCards,HomeRedCards,AwayGoals,AwayGoalsHalftime,AwayShots,AwayShotsOnTarget,AwayCorners,AwayFouls,AwayYellowCards,AwayRedCards
0,2010/11,2010-08-14,M Dean,Aston Villa,West Ham,HomeWin,HomeWin,3,2,23,...,1,0,0,0,12,2,7,15,2,0
1,2010/11,2010-08-14,P Dowd,Blackburn,Everton,HomeWin,HomeWin,1,1,7,...,2,0,0,0,17,12,3,14,1,0
2,2010/11,2010-08-14,S Attwell,Bolton,Fulham,Draw,Draw,0,0,13,...,1,0,0,0,12,7,8,13,3,0
3,2010/11,2010-08-14,M Clattenburg,Chelsea,West Brom,HomeWin,HomeWin,6,2,18,...,1,0,0,0,10,4,1,10,0,0
4,2010/11,2010-08-14,A Taylor,Sunderland,Birmingham,Draw,HomeWin,2,1,6,...,3,1,2,0,13,7,6,10,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,2017/18,2018-05-13,M Atkinson,Newcastle,Chelsea,HomeWin,HomeWin,3,1,16,...,0,0,0,0,6,2,2,10,1,0
3036,2017/18,2018-05-13,A Marriner,Southampton,Man City,AwayWin,Draw,0,0,8,...,3,0,1,0,13,2,12,10,1,0
3037,2017/18,2018-05-13,A Taylor,Swansea,Stoke,AwayWin,AwayWin,1,1,26,...,1,0,2,2,8,5,0,9,2,0
3038,2017/18,2018-05-13,C Pawson,Tottenham,Leicester,HomeWin,AwayWin,5,1,14,...,1,0,4,2,16,9,4,13,2,0


### Web scraping

Python Selenium is a web scraping library that allows us to launch to automate web browsing

We used this bot to go to https://www.premierleague.com/ to attain the match data for each match. 

Essentially, each match's data is put into a single URL, where https://www.premierleague.com/match/{match_number} contains the data for the {match_number}th match, where match_number is a unique identifier for each match (see code below) 

All of the code that is used to scrape the data is in the cell below. The website data is converted to text and will create a PLData{season_number}.csv on the same directory as this .ipynb file. 

***(todo: consider adding a video of web scraping into this notebook, to show that we actually did it)***

### **Do take note that the cell below will not run unless this machine has Chrome Webdriver and Selenium installed. If you wish to test this code, it is suggested that you run this code in your local machine's runtime enviroment, rather than in this Jupyter Notebook server. Remove the first line when ready to scrape.**

In [ ]:
"""
FULL SCRAPER CODE
"""

errorcode = """
Please ensure that all relevant libraries, chrome webdriver and selenium is installed. 
It is suggested to run this code in your local machine, using a Python virtual environment. 
Remove this code once done
"""
raise ImportError(errorcode)

import csv
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# These are the match numbers for the FIRST match of every season, to be put into the URL! Note: This identifier may change, subject to the Premier League website's descretion.
SEASON_FIRST_MATCHES = {
    '1011': 7087,
    '1112': 7467,
    '1213': 7864,
    '1314': 9231,
    '1415': 9611,
    '1516': 12115,
    '1617': 14042,
    '1718': 22342
}

driver = webdriver.Chrome(ChromeDriverManager().install())

# First we open up the very first 10/11 premier league match
url = "https://www.premierleague.com/match/7087"
driver.get(url)

# Accept the cookies
cookie_css_class = "js-accept-all-close"  # From inspecting element


def wait_till_element_appears(by, selector_value, clickable=True, time=2):
    try:
        if clickable:
            WebDriverWait(driver, time).until(
                expected_conditions.element_to_be_clickable(
                    (by, selector_value))
            )
        else:
            WebDriverWait(driver, time).until(
                expected_conditions.presence_of_element_located(
                    (by, selector_value))
            )
    except TimeoutException:
        print("request timed out. Try again")
    except:
        print("Something went wrong. Check the code again")


# Just wait until the cookies button shows up
wait_till_element_appears(By.CLASS_NAME, cookie_css_class, time=4)

# first, grab the button
cookie_button = driver.find_element_by_class_name(cookie_css_class)

# click on the button
cookie_button.click()

# For every season in the hash table,
for season in SEASON_FIRST_MATCHES:

    # create the csv file, naming it the season that we want
    with open(f'PLdata{season}.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, dialect='excel')

        # Add the column names
        column_names = [
            'Season',
            'HomeTeam',
            'AwayTeam',
            'HomePossession',
            'AwayPossession',
            'HomeTouches',
            'AwayTouches',
            'HomePasses',
            'AwayPasses',
            'HomeTackles',
            'AwayTackles',
            'HomeClearances',
            'AwayClearances',
            'HomeOffsides',
            'AwayOffsides'
        ]
        csvwriter.writerow(column_names)

        try:
            # Now, for every match number
            for match_number in range(SEASON_FIRST_MATCHES[season], SEASON_FIRST_MATCHES[season]+380):

                # open the webpage dedicated to that match
                url = f'https://www.premierleague.com/match/{match_number}'
                driver.get(url)

                # This is the stats button XPATH
                stats_button_xpath = "//li[@data-tab-index='2']"

                # wait for the stats button to appear
                wait_till_element_appears(By.XPATH, stats_button_xpath, time=4)

                # click on the stats button
                try:
                    stats_button = driver.find_element_by_xpath(stats_button_xpath)
                    stats_button.click()
                except NoSuchElementException:
                    print(
                        "The element you were looking for could not be found. Check your XPATH")

                # Extract the data by getting the table rows
                wait_till_element_appears(
                    By.TAG_NAME, selector_value="table", clickable=False)
                time.sleep(1) 
                table_rows = driver.find_elements_by_xpath(
                    "//th/a") + driver.find_elements_by_xpath("//td/p")

                # Write into our csv file
                csv_row = [
                    '20' + season[:2] + '/' + season[2:],  # season
                    table_rows[2].text,  # home team
                    table_rows[3].text,  # away team
                    table_rows[4].text,  # home possession
                    table_rows[6].text,  # Away possession
                    table_rows[13].text,  # Home touches
                    table_rows[15].text,  # away touches
                    table_rows[16].text,  # Home passes
                    table_rows[18].text,  # Away passes
                    table_rows[19].text,  # Home tackles
                    table_rows[21].text,  # away tackles
                    table_rows[22].text,  # home clearances
                    table_rows[24].text,  # Away clearances
                    table_rows[28].text,  # Home offsides
                    table_rows[30].text,  # Away offsides
                ]
                csvwriter.writerow(csv_row)

        except TimeoutException:
            print("There was a time out")

driver.quit()


### Translating the data to a single Dataframe 

The above scraper code will organise each .csv file according to season data. Thus, we import these csv files, and combine them with Nguyen's data to obtain the final, full dataset.
**Please ensure that the csv files are available in this notebook's directory before running the below code. If not, import all the cleaned data from final_data.csv in the cell below this one, called "the final dataset"** 

In [ ]:
season1011 = pd.read_csv('PLdata1011.csv')
season1112 = pd.read_csv('PLdata1112.csv')
season1213 = pd.read_csv('PLdata1213.csv')
season1314 = pd.read_csv('PLdata1314.csv')
season1415 = pd.read_csv('PLdata1415.csv')
season1516 = pd.read_csv('PLdata1516.csv')
season1617 = pd.read_csv('PLdata1617.csv')
season1718 = pd.read_csv('PLdata1718.csv')

result = pd.concat([season1011, season1112, season1213, season1314, season1415, season1516, season1617, season1718], ignore_index=True)
final_data = pd.merge(left = result, right = all_seasons_data, left_index = True, right_index = True).drop(columns = ['HomeTeam_y', 'AwayTeam_y', 'Season_y'])
final_data.head()